In [6]:
from bonito_pretrained import BonitoPretrained
model = BonitoPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000)

In [5]:
from dataloading2 import nanopore_datamodule
from data_utils.split_methods import get_kfold_split_func
dm = nanopore_datamodule(
    pos_files='pos_2022', 
    neg_files='neg_2022', 
    verbose=1, 
    batch_size=64, 
    valid_limit=10000, 
    workers=32,
    split_method=get_kfold_split_func(total_k=5, current_k=0),
    window=1000,
)

In [4]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

experiment_name = 'TEESTbonito_2022_kfold_0of5'
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints_pl/{experiment_name}", save_top_k=2, monitor="valid_loss")
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 5000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=10000,  
    log_every_n_steps=10000, benchmark=True, precision=16,
    callbacks=[checkpoint_callback])#, profiler="simple")

trainer.fit(model, dm)
